In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
fr = pd.DataFrame()

flights_to_add = ['N211LB', 'N226LB', 'N235LB', 'N238LB', 'N252LB', 'N271LB', 'N329LB', 'N789LB']

min_pts_list = ['4', '128', '256', '512', '1024', '1416']



In [3]:
for loon in flights_to_add:
    
    for mp in min_pts_list:

        aux = pd.read_csv('../../../../og_data/DBSCAN/' + 'eps0.1/' +  loon \
                          + '/CL-' + loon 
                          + '-mp_'+ str(mp) +  '.csv', \
                          parse_dates=['ts'], low_memory=False)

        aux['eps'] = 0.1
        aux['min_pts'] = mp
        aux['num_records'] = aux.shape[0]
        
        
        fr = fr.append(aux)

In [7]:
for loon in flights_to_add:
    
    for mp in min_pts_list:

        aux = pd.read_csv('../../../../og_data/DBSCAN/' + 'eps2.5/' +  loon \
                          + '/CL-' + loon 
                          + '-mp_'+ str(mp) +  '.csv', \
                          parse_dates=['ts'], low_memory=False)

        aux['eps'] = 0.00039
        aux['min_pts'] = mp
        aux['num_records'] = aux.shape[0]
        
        fr = fr.append(aux)

In [ ]:
for loon in flights_to_add:
    
    for mp in min_pts_list:

        aux = pd.read_csv('../../../../og_data/DBSCAN/' + 'eps6/' +  loon \
                          + '/CL-' + loon 
                          + '-mp_'+ str(mp) +  '.csv', \
                          parse_dates=['ts'], low_memory=False)

        aux['eps'] = 0.00094
        aux['min_pts'] = mp
        aux['num_records'] = aux.shape[0]
        
        fr = fr.append(aux)

In [ ]:
bl = fr

In [ ]:
bl

In [ ]:
cluster_stats_df = bl.groupby(['eps', 'min_pts', 'registration_id', 'cluster']).count().reset_index()

In [ ]:
cluster_stats_df

In [ ]:
# Let's join back the number of records of each flight
cluster_stats_df = pd.merge(cluster_stats_df, bl[['eps', 'min_pts', 'registration_id', 'cluster', 'num_records']], \
                 on=['eps', 'min_pts', 'registration_id', 'cluster'])

cluster_stats_df['num_records'] = cluster_stats_df['num_records_y']

cluster_stats_df = cluster_stats_df.drop('num_records_y', axis=1)
cluster_stats_df = cluster_stats_df.drop('num_records_x', axis=1)

cluster_stats_df['stationary'] = cluster_stats_df['cluster'].apply(lambda x: True if x != -1 else False)


cluster_stats_df


In [ ]:
for loon in flights_to_add:
    
    loon_cluster_stats = cluster_stats_df.loc[cluster_stats_df.registration_id == loon]

    loon_cluster_stats['share_of_data'] = loon_cluster_stats['ts']/loon_cluster_stats['num_records']

    loon_cluster_stats['min_pts'] = loon_cluster_stats['min_pts'].astype(float)
    loon_cluster_stats = loon_cluster_stats.sort_values('min_pts')
    loon_cluster_stats['min_pts'] = loon_cluster_stats['min_pts'].astype(str)
    
    plt.figure(loon, figsize=(30,6))

    ax = sns.lineplot(x=loon_cluster_stats['min_pts'], y="share_of_data", hue='stationary', sort=False,\
                     data=loon_cluster_stats)

    #ax.set(ylim=(0, 50))
    dummy = ax.set_ylabel('Share of Data')
    dummy = ax.set_xlabel('MinPts')
    dummy = ax.set_title(loon + ': DBSCAN cluster stats')
    dummy = ax.tick_params(labelsize=10)
    

In [ ]:
eps_stats_df = bl.groupby(['eps', 'min_pts', 'cluster']).count().reset_index()
eps_stats_df = eps_stats_df[['eps', 'min_pts', 'cluster']]

eps_stats_df = eps_stats_df.groupby(['eps', 'min_pts']).count().reset_index()

eps_stats_df

In [ ]:

eps_stats_df['cluster'] = eps_stats_df['cluster'] - 1

plt.figure(loon, figsize=(30,6))

eps_stats_df['min_pts'] = eps_stats_df['min_pts'].astype(float)
eps_stats_df = eps_stats_df.sort_values('min_pts')
eps_stats_df['min_pts'] = eps_stats_df['min_pts'].astype(str)

#eps_stats_df['eps'] = eps_stats_df['eps'].astype(str)

ax = sns.lineplot(x=eps_stats_df['min_pts'], y="cluster", hue=eps_stats_df['eps'].values, sort=False,\
                     data=eps_stats_df, palette = sns.color_palette("hls", 3))

#ax.set(ylim=(0, 50))
dummy = ax.set_ylabel('Number of clusters')
dummy = ax.set_xlabel('MinPts')
dummy = ax.set_title('Clusters generated for all loons')
dummy = ax.tick_params(labelsize=10)